<a href="https://colab.research.google.com/github/DamiamAlfaro/Sonder/blob/main/WebScraping/CSLBTraitLicenseCorrelation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from google.colab import files
#files.upload()
import openpyxl


In [2]:
url = "https://www.cslb.ca.gov/about_us/library/licensing_classifications/"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

links = soup.find_all('a', href=lambda href: href and "Licensing_Classifications_Detail.aspx?Class=" in href)

hrefs = [link['href'] for link in links]

licenseTexts = []

for license in hrefs[4:]:
  licensePage = url + license
  linkResponse = requests.get(licensePage)
  souping = BeautifulSoup(linkResponse.content, 'html.parser')
  section = souping.find("div", class_="main-primary")
  paragraphs = section.find_all("p")
  text = [paragraph.get_text() for paragraph in paragraphs]
  licenseTexts.append(text)

In [3]:
cleanedLicenseTexts = []

for licenseText in licenseTexts:
  cleansing = []
  for paragraph in licenseText:
    cleanedString = paragraph.replace("\n", "").replace("\t", "").replace("\r", "").replace("            "," ")
    cleansing.append(cleanedString)
  cleanedLicenseTexts.append(cleansing)

In [111]:
soup2 = BeautifulSoup(response.content, 'html.parser')

linksSpecialty = soup2.find('a',href="C-61_Limited_Speciality/Default.aspx")

specialties = url + linksSpecialty['href']

response2 = requests.get(specialties)
souping2 = BeautifulSoup(response2.content, 'html.parser')

# Method 1: All hrefs in Page
hrefs2 = souping2.find_all('a', href=lambda href: href and "Licensing_Classifications_Detail.aspx?Class=" in href)

# Method 2: Specific locations
ulClass = souping2.find('ul',class_='list-understated')

listElements = ulClass.find_all('li')

linksforEach = []

for element in listElements:
  filter = []
  for finiteElement in element.find_all('a'):
    if finiteElement['href'][0:2] == "..":
      filter.append(finiteElement['href'][2:])
    else:
      filter.append(finiteElement['href'])
  linksforEach.append(filter)

# Text Titles only
textlistElements = [element.get_text() for element in listElements]

specialtiesTitlesDescriptions = []

for x, y in zip(linksforEach, textlistElements):
  transportingStrings = []
  transportingStrings.append(y)
  for j in range(len(x)):
    direction = url + x[j]
    response3 = requests.get(direction)
    souping3 = BeautifulSoup(response3.content, 'html.parser')
    classing = souping3.find("div", class_="main-primary")
    try:
      paragraphs = classing.find_all("p")
    except AttributeError:
      continue
    textSpecialty = [paragraph.get_text() for paragraph in paragraphs]
    cleansing2 = []
    for paragraph in textSpecialty:
      cleanedString = paragraph.replace("\n", "").replace("\t", "").replace("\r", "")
      cleanFurther = " ".join(cleanedString.split())
      cleansing2.append(cleanFurther)
    transportingStrings.append(cleansing2)
  specialtiesTitlesDescriptions.append(transportingStrings)


In [150]:
specialtiesExcelTransition = []

for specialtyCleaned in specialtiesTitlesDescriptions:
  agentTransport = []
  for specialtyAttribute in specialtyCleaned:
    if type(specialtyAttribute) == list:
      for attributeItself in specialtyAttribute:
        agentTransport.append(attributeItself)
    else:
      cleanedString = specialtyAttribute.replace("-", "")
      anotherClean = cleanedString.split("  ")
      agentTransport.append(anotherClean[0])
      try:
        agentTransport.append(anotherClean[1])
      except IndexError:
        continue
  specialtiesExcelTransition.append(agentTransport)

In [151]:
"""
TRANSCRIPTION OCCURRED LOCALLY
"""

['D1', 'Architectural Porcelain (Now under D64)', 'California Code of Regulations Title 16, Division 8, Article 3. Classifications', 'A nonspecialized contractor installs, modifies, maintains and repairs new products and/or new installations which are not defined in any section herein or defined in any license classification authorized by the Board under Chapter 9, Division 3 of the Business and Professions Code (Contractors License Law).']
['D2', 'Asbestos Fabrication (Now under C2)', 'California Code of Regulations Title 16, Division 8, Article 3. Classifications', 'An insulation and acoustical contractor installs any insulating media and preformed architectural acoustical materials for the purpose of temperature and/or sound control.', 'Authority cited: Sections 7008 and 7059, Reference: Sections 7058 and 7059 (Business and Professions Code)']
['D3', 'Awnings', 'California Code of Regulations Title 16, Division 8, Article 3. Classifications', 'An awning contractor installs, modifies